# Installing Libraries

In [1]:
%%sh

sudo su 

#Download appropriate package for the OS version
#Choose only ONE of the following, corresponding to your OS version

#RHEL 7 and Oracle Linux 7
curl https://packages.microsoft.com/config/rhel/7/prod.repo > /etc/yum.repos.d/mssql-release.repo

#RHEL 8 and Oracle Linux 8
#curl https://packages.microsoft.com/config/rhel/8/prod.repo > /etc/yum.repos.d/mssql-release.repo

#RHEL 9
#curl https://packages.microsoft.com/config/rhel/9.0/prod.repo > /etc/yum.repos.d/mssql-release.repo

exit

sudo yum remove unixODBC-utf16 unixODBC-utf16-devel #to avoid conflicts
sudo ACCEPT_EULA=Y yum install -y msodbcsql17
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y yum install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo yum install -y unixODBC-devel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   413  100   413    0     0   7592      0 --:--:-- --:--:-- --:--:--     0 --:--:-- --:--:--  7648


Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


No Match for argument: unixODBC-utf16
No Match for argument: unixODBC-utf16-devel


No Packages marked for removal
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


67 packages excluded due to repository priority protections
Package msodbcsql17-17.10.6.1-1.x86_64 already installed and latest version
Nothing to do
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


67 packages excluded due to repository priority protections
Package mssql-tools-17.10.1.1-1.x86_64 already installed and latest version
Nothing to do
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


67 packages excluded due to repository priority protections
Package unixODBC-devel-2.3.1-15.amzn2.x86_64 already installed and latest version
Nothing to do


# Installing libraries

In [2]:
!conda install polars -y -q

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



# Connecting to database

In [33]:
import re
import pyodbc
import requests

In [18]:
file_path = '/home/ec2-user/SageMaker/db-credentials.txt'

with open(file_path, 'r') as file:
    # Code to parse the data will go here
    file_contents = file.read()

# Remove newlines and extra spaces
cleaned_string = file_contents.replace('\n', '').strip()

# Extract variable-value pairs using regular expressions
pattern = r'"([^"]+)": "([^"]+)"'
pairs = re.findall(pattern, cleaned_string)

parsed_data = {}

for variable, data in pairs:
    parsed_data[variable] = data

In [19]:
### This example shows how to view tables in the AAOCHIN2023 database. 
raw_db = 'AAOCHIN2023' 
raw_connection_string = "DRIVER={ODBC Driver 17 for SQL Server};" + \
                    "SERVER=" + parsed_data['host'] + ',' + parsed_data['port'] + ';' + \
                    "DATABASE=" + raw_db + ';' + \
                    "UID=" + parsed_data['username'] + ';' + \
                    "PWD={" + parsed_data['password'] + "};"
raw_conn = pyodbc.connect(raw_connection_string, trusted_connection = 'no')
raw_cursor = raw_conn.cursor()

In [20]:
#### CHECK IF DATABASE CURSOR IS WORKING
raw_cursor.execute('''
SELECT name AS TABLE_NAME
FROM sys.views
''')
for row in raw_cursor:
    print(row[0])

OBSERVATION_FACT
PATIENT_DIMENSION
PROVIDER_DIMENSION
VISIT_DIMENSION
ZCTA_CVS
SIDECAR_FACILITIES
CONCEPT_DIMENSION
I2B2


# Importing required libraries

In [10]:
import polars as pl
import numpy as np

# Data

In [11]:
Outcome0_pids_df = pl.read_parquet('../Determine_cohort_after_visit_index_details.parquet')
Outcome0_pids_df.head()

PATIENT_NUM,Encounter_Count,first_encounter_date,Last_encounter_date,FirstOutcomeDate,Index_Start_date,DEATH_DATE
i64,i64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
8717144,46,2020-02-10 00:00:00,2022-12-28 00:00:00,null,2019-04-08 00:00:00,null
8900099,3,2020-11-12 00:00:00,2021-07-15 00:00:00,null,2019-02-23 00:00:00,null
7452088,23,2019-05-10 00:00:00,2022-08-25 00:00:00,null,2019-01-29 00:00:00,null
576517,49,2019-05-28 00:00:00,2022-07-26 00:00:00,null,2019-04-18 00:00:00,null
9958038,10,2019-05-13 00:00:00,2022-12-07 00:00:00,null,2018-09-27 00:00:00,null


In [12]:
pids_to_focus = Outcome0_pids_df['PATIENT_NUM'].to_list()

### Get active ingridents after the index visit

In [34]:
def get_active_ingredient(rxcui):
       # Define the base URL of the RxNorm API
       base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
       
       # Create the complete URL for the request
       request_url = f"{base_url}{rxcui}/related.json?tty=IN"
       
       try:
           # Make the GET request to the API
           response = requests.get(request_url)
           response.raise_for_status()  # Raise an error if the request was unsuccessful
           print(response)
           # Parse the JSON response
           data = response.json()
#            print(data)
           # Extract the active ingredient from the JSON response
           ingredients = data.get('relatedGroup', {}).get('conceptGroup', [])
           for group in ingredients:
               if group.get('tty') == 'IN':  # Here we're looking for ingredients
                   active_ingredients = [concept['name'] for concept in group.get('conceptProperties', [])]
                   return active_ingredients
       
       except requests.RequestException as e:
           print(f"An error occurred: {e}")

       return []

In [16]:
patient_nums_str = ', '.join(map(str, pids_to_focus))

In [21]:
test_df = pl.read_database(f""" 
SELECT * from S348.dbo.patient_records_after_index_visit_medications""", connection=raw_cursor)



In [22]:
test_df.write_parquet('Determine_medications_after_index_visit_for_Outcome0.parquet')

In [29]:
import pickle
## Loading mapping of rxnorm to active ingridents list
with open('../rxnorm_active_ingridents_mapping.pkl', 'rb') as file: 
    dict_rxnorm_active_ing = pickle.load(file)

In [41]:
### mapping rxnorm to active ingredient in medications datafrane ###
no_act_ing_codes = []
rxnorms_dataframe_list = test_df.select(['RxNorm'])['RxNorm'].to_list()
active_ingredients_list_dataframe = []
for rxnorm in rxnorms_dataframe_list:
    rx_code = rxnorm.split(':')[-1].strip()
    try:
        if dict_rxnorm_active_ing[rx_code] == []:
            no_act_ing_codes.append(rx_code)
            active_ingredients_list_dataframe.append([str(rx_code)]) ### handling codes that dont have active ingredient associated with them
            continue
        active_ingredients_list_dataframe.append(dict_rxnorm_active_ing[rx_code])
    except:
        active_ingredients_list_dataframe.append(get_active_ingredient(rx_code))
        dict_rxnorm_active_ing[rx_code] = get_active_ingredient(rx_code)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [36]:
len(np.unique([rx.split(':')[-1].strip() for rx in rxnorms_dataframe_list]))

6208

In [42]:
len(rxnorms_dataframe_list)

23995175

In [50]:
rxnorms_dataframe_list[:5]

['RXNORM:1000001 ',
 'RXNORM:1000001 ',
 'RXNORM:1000001 ',
 'RXNORM:1000001 ',
 'RXNORM:1000001 ']

In [43]:
len(active_ingredients_list_dataframe)

23995175

In [44]:
len(no_act_ing_codes)

153944

In [63]:
## ignoring medications shared by domain expert
ignore_rxnorm_codes = [
    "16681", "173", "1534763", "1368001", "1368384", "1368402", 
    "2627044", "1373458", "1545149", "2404", "1488564", "1486436", 
    "1727500", "1551291", "1545653", "1598392", "2281864", "1664314", 
    "1992672", "1992684", "1992825", "60548", "4816", "25789", 
    "647235", "606253", "4821", "352381", "25793", "4815", 
    "285129", "1007184", "51428", "1670007", "1727493", "139825", 
    "274783", "1858994", "400008", "631657", "1605101", "1008501", 
    "86009", "816726", "253182", "1100699", "1243019", "475968", 
    "1440051", "6809", "607999", "802646", "614348", "1043562", 
    "729717", "30009", "274332", "33738", "139953", "73044", 
    "84108", "857974", "1991302", "593411", "2638675", "2621880", 
    "2601723", "10633", "10635", "72610", "596554"
]

#medications_df = medications_df.filter(~pl.col('rxcui').is_in(ignore_rxnorm_codes))
act_list_to_drop = [get_active_ingredient(rxcui) for rxcui in ignore_rxnorm_codes]
medications_to_drop = ["_".join(sublist) if len(sublist) > 1 else sublist[0] for sublist in act_list_to_drop]

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [56]:
active_ingredients_list_dataframe

[['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan', 'hydrochlorothiazide'],
 ['amlodipine', 'olmesartan'

In [59]:
final_act_ing_list = []
for sublist in active_ingredients_list_dataframe:
    if len(sublist) > 1:
        final_act_ing_list.append("_".join(sublist))
    else:
        try:
            final_act_ing_list.append(sublist[0])
        except:
            final_act_ing_list.append('No act got')
            
print(len(final_act_ing_list))

23995175


In [60]:
def get_rx_code(element):
    return element.split(':')[-1].strip()

In [61]:
## Adding active ingident data to dataframe
test_df = test_df.with_columns(pl.Series(name="Active_ingrident", values=final_act_ing_list),
                                            pl.col('RxNorm').map_elements(get_rx_code,return_dtype = pl.Utf8).alias('rxcui'))

In [65]:
med_ignore_patient_num = np.unique(test_df.filter(pl.col('Active_ingrident').is_in(medications_to_drop))['PATIENT_NUM'].to_list())
print("Number of patients to ignore: ",len(med_ignore_patient_num))

Number of patients to ignore:  22651


In [67]:
pids_no_enc_after_vi = pl.read_parquet("../Determine_cohort_after_visit_index_details.parquet")

In [68]:
pids_no_enc_after_vi.head()

PATIENT_NUM,Encounter_Count,first_encounter_date,Last_encounter_date,FirstOutcomeDate,Index_Start_date,DEATH_DATE
i64,i64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
8717144,46,2020-02-10 00:00:00,2022-12-28 00:00:00,null,2019-04-08 00:00:00,null
8900099,3,2020-11-12 00:00:00,2021-07-15 00:00:00,null,2019-02-23 00:00:00,null
7452088,23,2019-05-10 00:00:00,2022-08-25 00:00:00,null,2019-01-29 00:00:00,null
576517,49,2019-05-28 00:00:00,2022-07-26 00:00:00,null,2019-04-18 00:00:00,null
9958038,10,2019-05-13 00:00:00,2022-12-07 00:00:00,null,2018-09-27 00:00:00,null


In [72]:
pids_no_enc_after_vi.filter(pl.col('PATIENT_NUM').is_in(pids_to_focus) & (pl.col('Encounter_Count')==0))

PATIENT_NUM,Encounter_Count,first_encounter_date,Last_encounter_date,FirstOutcomeDate,Index_Start_date,DEATH_DATE
i64,i64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
